In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re
import os

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.io.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (15701 Collins Ave #3805, North Miami Beach, F...
2     (6901 Collins Ave #601, Miami Beach, FL 33141,...
3     (4775 Collins Ave APT 3503, Miami Beach, FL 33...
4     (9401 Collins Ave #1002, Surfside, FL 33154, U...
5     (5775 Collins Ave, Miami Beach, FL 33140, USA,...
6     (1000 Venetian Way APT 602, Miami Beach, FL 33...
7     (1000 Venetian Way APT 1004, Miami Beach, FL 3...
8     (848 Brickell Key Dr APT 905, Miami, FL 33131,...
9     (16385 Biscayne Blvd Unit 1207, Aventura, FL 3...
10    (650 West Ave APT 3012, Miami Beach, FL 33139,...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Ritz-Carlton Sunny Isles\r15701 Collins Ave 38...,12/27/2022,28,"$7,300,000.00",3204,"$2,278.40",Chad Carroll,Compass Florida LLC,Brian Judge,Compass Florida LLC,Ritz-Carlton Sunny Isles 15701 Collins Ave 380...,Ritz-Carlton Sunny Isles,15701 Collins Ave 3805 Sunny Isles Beach,"(15701 Collins Ave #3805, North Miami Beach, F...","(25.9219798, -80.1217032, 0.0)",25.921980,-80.121703,0.0
2,L'Atelier Condo\r6901 Collins Ave 601\rMiami B...,12/29/2022,28,"$4,500,000.00",2133,"$2,109.70",Ron Laviv,One Sotheby's International Re,Ron Laviv,One Sotheby's International Re,L'Atelier Condo 6901 Collins Ave 601 Miami Beach,L'Atelier Condo,6901 Collins Ave 601 Miami Beach,"(6901 Collins Ave #601, Miami Beach, FL 33141,...","(25.8544637, -80.1200027, 0.0)",25.854464,-80.120003,0.0
3,Green Diamond\r4775 Collins Ave 3503\rMiami Beach,12/28/2022,141,"$3,000,000.00",1980,"$1,515.15",Menachem Fellig,Compass Florida LLC,Joanna Jimenez,Compass Florida LLC,Green Diamond 4775 Collins Ave 3503 Miami Beach,Green Diamond,4775 Collins Ave 3503 Miami Beach,"(4775 Collins Ave APT 3503, Miami Beach, FL 33...","(25.8227293, -80.1226745, 0.0)",25.822729,-80.122675,0.0
4,Azure Condo\r9401 Collins Ave 1002\rSurfside,12/27/2022,4,"$2,276,000.00",1945,"$1,170.18",Ivan Chorney,Compass Florida LLC,Klaudia Juniewicz,Engel & Voelkers Coconut Grove,Azure Condo 9401 Collins Ave 1002 Surfside,Azure Condo,9401 Collins Ave 1002 Surfside,"(9401 Collins Ave #1002, Surfside, FL 33154, U...","(25.883813, -80.1217561, 0.0)",25.883813,-80.121756,0.0
5,57 Ocean\r5775 Collins Ave 605\rMiami Beach,12/28/2022,89,"$2,225,000.00",1754,"$1,787.15",Dina Goldentayer,Douglas Elliman,Christopher Jude,Fortune Christie's Int'l Real Estate,57 Ocean 5775 Collins Ave 605 Miami Beach,,57 Ocean 5775 Collins Ave 605 Miami Beach,"(5775 Collins Ave, Miami Beach, FL 33140, USA,...","(25.8392568, -80.12039469999999, 0.0)",25.839257,-80.120395,0.0
6,One Thousand Venetian\r1000 Venetian Way 602\r...,12/27/2022,44,"$2,150,000.00",2010,"$1,069.65",Anna Sherrill,One Sotheby's International Re,Selim Ozutez,LoKation,One Thousand Venetian 1000 Venetian Way 602 Miami,One Thousand Venetian,1000 Venetian Way 602 Miami,"(1000 Venetian Way APT 602, Miami Beach, FL 33...","(25.789686, -80.1739309, 0.0)",25.789686,-80.173931,0.0
7,One Thousand Venetian\r1000 Venetian Way 1004\...,12/27/2022,11,"$2,000,000.00",1720,"$1,162.79",Aouri Makhlouf,Compass Florida LLC,Anna Sherrill,One Sotheby's International Re,One Thousand Venetian 1000 Venetian Way 1004 M...,One Thousand Venetian,1000 Venetian Way 1004 Miami,"(1000 Venetian Way APT 1004, Miami Beach, FL 3...","(25.789686, -80.1739309, 0.0)",25.789686,-80.173931,0.0
8,Three Tequesta Point\r848 Brickell Key Dr 905\...,12/26/2022,58,"$1,030,000.00",1734,$594.00,Stephan Hermida,Engel & Voelkers Coconut Grove,Claudia Avellan-Juez,Korman Realty LLC,Three Tequesta Point 848 Brickell Key Dr 905 M...,Three Tequesta Point,848 Brickell Key Dr 905 Miami,"(848 Brickell Key Dr APT 905, Miami, FL 33131,...","(25.7693285, -80.1837717, 0.0)",25.769329,-80.183772,0.0
9,The Harbour North Condo\r16385 Biscayne Blvd 1...,12/28/2022,385,"$900,000.00",1557,$578.03,Simy Chocron Morely,Citadel Realty,Gustavo Gambino,Gambino Real Estate Corp.,The Harbour North Condo 16385 Biscayne Blvd 12...,The Harbour North Condo,16385 Biscayne Blvd 1207 North Miami Beach,"(16385 Biscayne Blvd Unit 1207, Aventura, FL 3...","(25.9285885, -80.1509559, 0.0)",25.928589,-80.150956,0.0
10,0The Floridian\r650 West Ave 3012\rMiami Beach,12/28/2022,1,"$900,000.00",1147,$784.66,Dimitri Voncini,Compass Florida LLC,Dimitri Voncini,Compass Florida LLC,0The Floridian 650 West Ave 3012 Miami Beach,,0The Floridian 650 West Ave 3012 Miami Beach,"(650 West Ave APT 3012, Miami Beach, FL 33139,...","(25.7764662, -80.1421986, 0.0)",25.77

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.at[3,'Agent']=('Mendel Fellig')

df.at[5,'building_name']=('57 Ocean')
df.at[5,'address_only']=('5775 Collins Ave 605 Miami Beach')

df.at[10,'building_name']=('The Floridian')
df.at[10,'address_only']=('650 West Ave 3012 Miami Beach')

In [11]:
df.columns

Index(['building_address_city', 'Close Date', 'days_on_market', 'Sale Price',
       'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker', 'Buyer Agent',
       'Buyer Broker', 'geo_address', 'building_name', 'address_only', 'loc',
       'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [12]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=10)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('December 25th - December 31th')

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [13]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [14]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [15]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_123122
